# Extraction

In [1]:
using LinearAlgebra, BenchmarkTools, SparseArrays, Plots
cd("/Users/junwong/Dropbox/Second Year/Kellogg - Environmental/Assignments")

# 1. Discrete state space

## Construct state & action spaces

In [2]:
S = collect(0:2:1000);
N = size(S,1);
A = collect(0:2:1000); # Action space is how many units I can extract, let this be the same as the state space 
δ = 1/1.05;
indexed_states = zeros(size(S,1), size(A,1));


## Define utility functions

In [3]:
U1(x) = 2 * x^0.5 ;
MU1(x) = x ^ -0.5;
U2(x) = 5 * x - 0.05 * (x ^ 2);
MU2(x) = 5 - 0.1 * x;

## Flow utility
Constructs a $501 \times 501$ matrix of flow utility given some utility function defined above

In [4]:
find_flow = function(ufn)
    flow_U = zeros(size(S,1), size(A,1));
    for i in 1:size(S,1) #stock 
        for j in 1:size(A,1) #extraction
            if j <= i 
                flow_U[i,j] = ufn(A[j])
            else 
               flow_U[i,j] = -99999 # if j > i, i.e. extraction > stock, utility is -inf  
            end
        end
    end
    
    return flow_U
end


#1 (generic function with 1 method)

## Find index of state in $t+1$ given action in $t$

In [5]:
state_index_t1 = zeros(size(S,1), size(A,1))

for i in 1:size(S,1)
    for j in 1:size(A,1)
        if j <= i
            left = S[i] - A[j]
            state_index_t1[i,j] = findall(x->x==left, S)[1]
        else
            state_index_t1[i,j] = 1 # it will be stock = 0 if I try to extract more than I have
         end
    end
end


## Construct transition matrix
The sparse matrix is created by pushing the row and column indices with nonzero entries. There's definitely a better way of doing this rather than looping over all rows and columns. There's also a smarter way to get the correct column index without creating this `colindex` item and have it clunk up memory...

In [6]:
# you need to find the state + action index of each column index
colindex = []
iter = 0 
for j in 1:size(A, 1)
    for i in 1:size(S,1)
        iter += 1 
        push!(colindex, [iter, i, j])
    end
end

# i,j = 1 iff state_index_t1 says I transition to j given action A 
# (this is a horrible way to constructing my sparse matrix but whatever)
v = Float64[]
r = []
c = []
for i in 1:size(S,1)
    for j in 1:size(colindex,1)
        if state_index_t1[i,colindex[j][3]] == colindex[j][2]
            push!(r, i)
            push!(c, colindex[j][1])
            push!(v, 1)
        end
    end
end

In [7]:
T = sparse(r, c, v)

501×250501 SparseMatrixCSC{Float64, Int64} with 251001 stored entries:
⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛

## Iterating function
The Bellman operator here is: $$ TV(S) = \max_{a \in A} u(a) + \delta E[V(S')] $$ such that $S' = S - a$

In [8]:
iterate_wrapper = function(utility_fn, discount, tol, transition, S, A)
    U = find_flow(utility_fn)
    global V = zeros(size(S,1))
    δ = discount
    err = 1
    iter = 0
    
    while err > tol
        iter += 1
        V_old = copy(V)
        
        # calculate v_next given each action
        Vₙₑₓₜ = zeros(size(S,1), size(A,1))
        for x in 1:size(A,1)-1
            Vₙₑₓₜ[:,x] = transition[:, 1+size(S,1) * (x-1):501+size(S,1) * (x-1)] * V_old
        end
        
        # find optimal action over each state and action 
        global V = maximum(U + δ .* Vₙₑₓₜ, dims=2) 
        global gₖ = argmax(U + δ .* Vₙₑₓₜ, dims=2)

        #println(maximum(abs.(v .- v_old)))
        err = maximum(abs.(V .- V_old))
    end
    
    return [V, gₖ, err, iter]
end

#5 (generic function with 1 method)

## Actually answering Q1

Find optimal control

In [9]:
results1 = iterate_wrapper(U1, δ, 1e-5, T, S, A);
results2 = iterate_wrapper(U2, δ, 1e-5, T, S, A);

Simulate extraction and price path

In [10]:
simulate_paths = function(oc, S, A, initial, time, utility)
    stock = initial
    df = zeros(time, 3)
    
    for t=1:time
        state_i = findall(x->x==stock, S)[1]
        action_j = oc[state_i][2]
        stock = stock - A[action_j]
        price = utility(A[action_j])
        df[t, :] = [t, A[action_j], price]
    end
    
    plot(df[:,2], legend=false, ylabel="Extraction", xlabel="Time", 
        linewidth=1.2)
    savefig("output/a2_q1_" * string(utility) *  "_extraction.pdf");
    plot(df[:,3], legend=false, ylabel="Prices", xlabel="Time", 
        color="darkorange", linewidth=1.2)
    savefig("output/a2_q1_" * string(utility) * "_prices.pdf");
    
end

#7 (generic function with 1 method)

Plotting paths

In [11]:
# Change default font
plot_font = "Computer Modern"
default(fontfamily=plot_font)

In [12]:
# U1
simulate_paths(results1[2], S, A, 1000, 80, MU1);
simulate_paths(results2[2], S, A, 1000, 80, MU2);

# 2. Interpolating between states

## Redefine our state & action space

In [13]:
S = collect(0:2:1000);
A = collect(range(0, sqrt(1000), 501)).^2;


## Find state in $t+1$ given state in $t$
For each current state and action pair, push the index to the left and right of the "in between" state in $t+1$ and also the weight on the right index.

In [14]:
state_index_t1 = Array{Array}(undef, size(S,1), size(A,1))

for i in 1:size(S,1)
    for j in 1:size(A,1)
        if A[j] <= S[i]
            left = S[i] - A[j]
            # if action lines up exactly with a state 
            if length(findall(x->x==left, S)) != 0 
                state_index_t1[i,j] = [findall(x->x==left, S)[1]]
            # try to extract the closest state space on the left and right
            else 
                pos = findfirst(S.-left .>= 0)
                neg = pos - 1
                
                # get the weights since (this is for the right side)
                weight = abs(left - S[pos])/abs(S[pos]-S[neg])
                state_index_t1[i,j] = [neg, pos, weight]
            end 
        else
            state_index_t1[i,j] = [1] # it will be stock = 0 if I try to extract more than I have
        end
    end
end


## Construct transition matrix
Given the left & right index and the weighting, simply push if the indices match up in this transition matrix

In [15]:
# Get index of state and action given column number
colindex = []
iter = 0 
for j in 1:size(A, 1)
    for i in 1:size(S,1)
        iter += 1 
        push!(colindex, [iter, i, j])
    end
end

# Transition
v = Float64[]
r = []
c = []
for i in 1:size(S,1)
    for j in 1:size(colindex,1) 
        if length(state_index_t1[i,colindex[j][3]])==1
            if state_index_t1[i,colindex[j][3]][1] == colindex[j][2]
                push!(r, i)
                push!(c, colindex[j][1])
                push!(v, 1)
            end
        else 
            if state_index_t1[i,colindex[j][3]][1] == colindex[j][2]
                push!(r, i)
                push!(c, colindex[j][1])
                push!(v, state_index_t1[i,colindex[j][3]][3])
            elseif state_index_t1[i,colindex[j][3]][2] == colindex[j][2]
                push!(r, i)
                push!(c, colindex[j][1])
                push!(v, 1-state_index_t1[i,colindex[j][3]][3])
            end
        end
    end
end

In [16]:
T = sparse(r, c, v)

501×250501 SparseMatrixCSC{Float64, Int64} with 415178 stored entries:
⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛

## Actually answering Q2
The iterating wrapper remains the same from Q1, we just have a different transition matrix in this instance.

In [17]:
result1 = iterate_wrapper(U1, δ, 1e-8, T, S, A);
result2 = iterate_wrapper(U2, δ, 1e-8, T, S, A);

Simulate time paths

In [18]:
simulate_paths = function(oc, S, A, initial, time, utility) 
    stock = initial 
    df=zeros(time, 3)
    for t in 1:time
        # get index and weight as above 
        pos = findfirst(S .- stock .>= 0)
        neg = pos - 1 
        weight = abs(stock - S[neg])/abs(S[pos] - S[neg])
        
        # calculate the weighted optimal action given the state we are in
        action = weight * A[oc[pos][2]] + (1-weight) * A[oc[neg][2]]
        stock = stock - action
        price = utility(action)
        df[t,:] = [t, action, price]
    end
    
    plot(df[:,2], legend=false, ylabel="Extraction", xlabel="Time", 
        linewidth=1.2)
    savefig("output/a2_q2_" * string(utility) *  "_extraction.pdf");
    plot(df[:,3], legend=false, ylabel="Prices", xlabel="Time", 
        color="darkorange", linewidth=1.2)
    savefig("output/a2_q2_" * string(utility) * "_prices.pdf");
end;    
    

In [22]:
simulate_paths(result1[2], S, A, 1000, 80, MU1);
simulate_paths(result2[2], S, A, 1000, 80, MU2);